In [ ]:
from lib.lbGraficas import cargaDataSet
from lib.lbGraficas import showGraficaCont
from lib.lbClima import cargaDfClima
from lib.lbClima import quitarHora
from lib.lbVarios import series_to_supervised
from lib.lbCargaDatosNorm import cargaDatosSep
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from math import sqrt
from sklearn.metrics import mean_squared_error
from os import path
import numpy
import pandas

In [ ]:
"""Ya no hace falta esta carga"""
#dfSol = cargaDatosSep('datos/datosContaminacion/datos17.csv',';')
#print ("=======")
#print (dfSol)
#print (dfSol.shape)

In [ ]:
import time
import datetime
hInicio = time.strftime("%H-%M-%S")
#print (time.strftime("%Y-%m-%d_%H-%M-%S"))
# load the network weights
#filepath="checkpoint/weightsV1.hdf5"
#model.load_weights(filepath) #Descomentar cuando tengamos un checkpoint
#fecha_archivos = time.strftime("%Y-%m-%d_%H-%M-%S")
#fecha_archivos = time.strftime("%Y-%m-%d_%H)
#print(fecha_archivos)

In [ ]:
""" -------------------- load dataset -------------------- """
dataset14 = cargaDataSet("datos/datosContaminacion/datos2014_DatosDiarios.txt") #datos 2014
dataset15 = cargaDataSet("datos/datosContaminacion/datos2015_DatosDiarios.txt") #datos 2015
dataset16 = cargaDataSet("datos/datosContaminacion/datos2016_DatosDiarios.txt") #datos 2016
#dataset = pandas.concat([dataset15,dataset16])
#dataset = pandas.concat([dataset14,dataset15,dataset16])
#print(dataset.head())
print(dataset15.shape)
print(dataset16.shape)

In [ ]:
#Para 2017 y 2018 cambia el formato del txt, entonces lo cogemos en csv ya directamente
dataset17 = pandas.read_csv('datos/datosContaminacion/datos17.csv',sep=';')
df17 = dataset17.drop(['V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10','V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20','V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31'], axis=1)
df17['Tecnica']='00'
df17['Periodo_Analisis']='04'
df17Filtrado = df17.loc[df17['ESTACION'] == 4]
df17Filtrado['ESTACION']='28079004'
dataset17 = df17Filtrado[['ESTACION','MAGNITUD','Tecnica','Periodo_Analisis','ANO','MES','D01','D02','D03','D04','D05','D06','D07','D08','D09','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20','D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']]
#dataset17 = df17Filtrado.loc['ESTACION','MAGNITUD','Tecnica','Periodo_Analisis','ANO','MES','D01','D02','D03','D04','D05','D06','D07','D08','D09','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20','D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']
dataset17.columns = ['Estacion','Magnitud','Tecnica','Periodo_Analisis','Año','Mes',"Dia1","Dia2","Dia3","Dia4","Dia5","Dia6","Dia7","Dia8","Dia9","Dia10","Dia11","Dia12","Dia13","Dia14","Dia15","Dia16","Dia17","Dia18","Dia19","Dia20","Dia21","Dia22","Dia23","Dia24","Dia25","Dia26","Dia27","Dia28","Dia29","Dia30","Dia31"]
print(dataset17.shape)

dataset18 = pandas.read_csv('datos/datosContaminacion/datos18.csv',sep=';')
df18 = dataset18.drop(['V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10','V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20','V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31'], axis=1)
df18['Tecnica']='00'
df18['Periodo_Analisis']='04'
df18Filtrado = df18.loc[df18['ESTACION'] == 4]
df18Filtrado['ESTACION']='28079004'
dataset18 = df18Filtrado[['ESTACION','MAGNITUD','Tecnica','Periodo_Analisis','ANO','MES','D01','D02','D03','D04','D05','D06','D07','D08','D09','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20','D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']]
#dataset18 = df18Filtrado.loc['ESTACION','MAGNITUD','Tecnica','Periodo_Analisis','ANO','MES','D01','D02','D03','D04','D05','D06','D07','D08','D09','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20','D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']
dataset18.columns = ['Estacion','Magnitud','Tecnica','Periodo_Analisis','Año','Mes',"Dia1","Dia2","Dia3","Dia4","Dia5","Dia6","Dia7","Dia8","Dia9","Dia10","Dia11","Dia12","Dia13","Dia14","Dia15","Dia16","Dia17","Dia18","Dia19","Dia20","Dia21","Dia22","Dia23","Dia24","Dia25","Dia26","Dia27","Dia28","Dia29","Dia30","Dia31"]
print(dataset18.shape)


In [ ]:
dataset = pandas.concat([dataset14,dataset15,dataset16,dataset17,dataset18])
print(dataset.shape)
plzEspana = "28079004"
print("ahora filtramos y colocamos todos los días en orden")
values08 = showGraficaCont(dataset,plzEspana,"08",False) #valor de NO2 en cada día
values8 = showGraficaCont(dataset,plzEspana,"8",True) #valor de NO2 en cada día
values01 = showGraficaCont(dataset,plzEspana,"01",False) #valor de SO2 en cada día
values1 = showGraficaCont(dataset,plzEspana,"1",True) #valor de SO2 en cada día
values06 = showGraficaCont(dataset,plzEspana,"06",False) #valor de CO en cada día
values6 = showGraficaCont(dataset,plzEspana,"6",True) #valor de CO en cada día
print("Número de valores de 08: " + str(len(values08)) + " Y número de valores de 8: " + str(len(values8)))
print("Número de valores de 01: " + str(len(values01)) + " Y número de valores de 1: " + str(len(values1)))
print("Número de valores de 06: " + str(len(values06)) + " Y número de valores de 6: " + str(len(values6)))
values08 = values08 + values8
values01 = values01 + values1
values06 = values06 + values6
print("Valor final de los conjuntos de valores 08,01 y 06:")
print(len(values08))
print(len(values01))
print(len(values06))

In [ ]:
print(numpy.amin(values08) )
print(numpy.amax(values08) )

In [ ]:
#rutaArchivosClima = path.dirname(path.abspath(__file__)) +'\datos\datosAemetJuntos'
#rutaArchivosClima = 'datos/datosAemetJuntos'
#rutaArchivosClima = 'datos/datosAemet141516'
rutaArchivosClima = 'datos/datosAemet14151617Abril18'
cabecerasClima = ['Estación','Provincia','Temperatura máxima (ºC)','Temperatura mínima (ºC)','Temperatura media (ºC)','Racha (km/h)','Velocidad máxima (km/h)','Precipitación 00-24h (mm)','Precipitación 00-06h (mm)','Precipitación 06-12h (mm)','Precipitación 12-18h (mm)','Precipitación 18-24h (mm)']
#dfClima = cargaDfClima(rutaArchivosClima,cabecerasClima,'Madrid, Retiro')
dfClima = cargaDfClima(rutaArchivosClima,cabecerasClima,'Madrid, Ciudad Universitaria')
#print(dfClima.head())
print(dfClima.shape)

In [ ]:
""" ---------- Construimos el dataset con los valores que queremos predecir ---------- """
valuesFinal= numpy.zeros((1,dfClima.shape[1]+1)) ##valuesFinal= numpy.zeros((1,13))
#dfClima = dfClima.fillna(0) #rellena con 0 los nulos - Cambiar a que coja lo del día de antes o después
#controlar que no haya nulos el primer dia
#Para poner el valor medio: df['A'].fillna(df['A'].median())
#Para limitarlo a que se haga en los primeros: df = df.fillna(value='missing', method='bfill', limit=1)
dfClima = dfClima.fillna(method='ffill') #rellena los nulos con el valor del día de antes
matrizClima = dfClima.reset_index().values
print(matrizClima.shape)
print(len(values08))
for i in range (0,len(values08),1):  
    newValueArr=numpy.array([[values08[i], values01[i], values06[i], quitarHora(matrizClima[i][3]), quitarHora(matrizClima[i][4]), matrizClima[i][5], quitarHora(matrizClima[i][6]), quitarHora(matrizClima[i][7]), matrizClima[i][8], matrizClima[i][9], matrizClima[i][10], matrizClima[i][11], matrizClima[i][12]]]).astype(float)     
    valuesFinal=numpy.append(valuesFinal, newValueArr, axis = 0)
print(valuesFinal[0])
valuesFinal = numpy.delete(valuesFinal, 0, axis=0) #Eliminamos el primero que inicializamos
print(valuesFinal[0])
print(len(valuesFinal))
#print(valuesFinal[730])
#print(valuesFinal[731])
print('===...===')
numpy.nan_to_num(valuesFinal,copy=False)
print(valuesFinal[0])
print('======')
#print(values08[730])
#print(values08[731])
print(values08[0])
#print('::::::::::::')
#print(valuesFinal[1000])
#print(valuesFinal[1095])

In [ ]:
""" -------------------- normalize features -------------------- """
#Hay que normalizar la matriz resultante para ello usamos MinMaxScaler() y scaler()
#scaler (son los valores normalizados) necesita el formato => Filas = Dias, Columnas = ValorCadaCosaQueMide
scaler_VF = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(valuesFinal)
scaler_fit = scaler_VF.fit(valuesFinal)
scaler_tra = scaler_VF.transform(valuesFinal)
scaled = scaler_tra
print(scaled.shape) #sera 366 + 365
print(scaled[0])
print('------')
print(scaled[1])
print('------')
print(scaled[2])

In [ ]:
""" preparamos los dos conjuntos (datosDiasPrevios,DiaSiguiente)"""
#n_chars = len(raw_text)
#nvocab = len(chars)
n_examples = len(valuesFinal)
# prepare the dataset of input to output pairs encoded as integers 
seq_length = 15 #coger sólo 15 días
dataX = [] 
dataY = [] 
for i in range(0, n_examples - seq_length, 1): 
    seq_in = scaled[i:i + seq_length] 
    seq_out = scaled[i + seq_length] 
    dataX.append(seq_in) 
    dataY.append(seq_out) 
n_patterns = len(dataX) 
print("Total Patterns: ", n_patterns)
print(len(dataX))
print(len(dataY))
#for j in range(0, 3):
#    print('====')
#    print(dataX[j])
#    print(':::::')
#    print(dataY[j])

In [ ]:
""" ::::::::::::::::::::  :::::::::::::::::::: """
#Train el 80% y Test el 20%
#Ahora hay 365 filas *2 y 26 columnas
#values = reframed.values
#n_train_days = round((values.shape[0]+1)*0.8)#292 #80% de 365
#n_train_days = round(730)#292 #80% de 365 si queremos 2 años de entrenamiento 730
n_train_days = round(1446)#292 #80% de 365 si queremos 2 años de entrenamiento 730
train_X=numpy.array(dataX[:n_train_days])
train_y=numpy.array(dataY[:n_train_days])
test_X=numpy.array(dataX[n_train_days:])
test_y=numpy.array(dataY[n_train_days:])

# reshape input to be 3D [samples, timesteps, features]
#train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
#test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
print('===')
#print(train_X[364])
#print(train_y[0])
#print(values[365][17
print(train_X.shape[1])
print(train_X.shape[2])

In [ ]:
#numpy.savetxt('test_X.csv', test_X, fmt='%.2f', delimiter=',')
#numpy.savetxt('test_X_No2.csv', values08, fmt='%.2f', delimiter=',')
#print(test_X.shape)
#te=[]
#print(test_X[0])
#print(test_X[0].shape)
#print(test_X.shape)
#print(test_y.shape)
#print(numpy.array(dataX[n_train_days]))
from keras.models import load_model
#model = load_model('my_model_2018-04-21_18-16-14.h5')
#loaded_model = load_model('checkpoint/weights2018-04-21_18-16-14.hdf5')
#desde que contamos con 14,15,16,17 para entrenar y 18 para test
#loaded_model = load_model('checkpoint/weights2018-05-22_02-39-19.hdf5')
#loaded_model = load_model('checkpoint/weights2018-05-23_09-19-56.hdf5

loaded_model = load_model('checkpoint/weights2018-06-02_21-19-29.hdf5')

# evaluate loaded model on test data 
# Define X_test & Y_test data first
#loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#score = loaded_model.evaluate(test_X, test_y, verbose=0)
#print ("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
lista_obtenido = []
lista_real = []
dias = 100
primer_dia = 0
variable_predicha = 0
error = 0


for i in range(primer_dia,primer_dia+dias):
    newPred = loaded_model.predict(train_X[i].reshape(1,15,13), verbose=0)
    real = train_y[i,variable_predicha]
    predicho = newPred[0,variable_predicha]
    error += abs(real-predicho)
    lista_real.append(real)
    lista_obtenido.append(predicho)

print("El error medio es: ", error / dias)
    
import matplotlib.pyplot as plt
grp = plt.figure(figsize=(20,5))
plt.plot(lista_real,"o-",color="blue",label="real")
plt.plot(lista_obtenido,"+-",color="red",label="estimado")
plt.legend()
plt.show()

In [ ]:
test_X[3,14]

In [ ]:
test_y[3]

In [ ]:
lista_obtenido = []
lista_real = []
dias = 100
primer_dia = 0
variable_predicha = 0
error = 0
desvTest = []

for i in range(primer_dia,primer_dia+dias):
    newPred = loaded_model.predict(test_X[i].reshape(1,15,13), verbose=0)
    real = test_y[i,variable_predicha]
    predicho = newPred[0,variable_predicha]
    error += abs(real-predicho)
    #print("El valor real es: ", real,' y el predicho es: ', predicho)
    desvTest.append(abs(real-predicho))
    lista_real.append(real)
    lista_obtenido.append(predicho)

print("El error medio es: ", error / dias)
    
import matplotlib.pyplot as plt
grp = plt.figure(figsize=(20,5))
plt.plot(lista_real,"o-",color="blue",label="real")
plt.plot(lista_obtenido,"+-",color="red",label="estimado")
plt.legend()
plt.show()

In [ ]:
numpy.set_printoptions(suppress=True)
newPred

In [ ]:
newPred = loaded_model.predict(test_X, verbose=1)

#desde = 1
#for d in range(0,351):
#    te.append(test_X[d][14][0])
#print(test_X[2][14][0])
#print("==")
#numpy.savetxt('test_X.csv', te, fmt='%.2f', delimiter=',')

import matplotlib.pyplot as plt
a=[]
b=[]

desde = 100
dias = 20
for dia in range(desde,desde+dias):
    a.append(test_X[dia-14][14][0])
    b.append(newPred[dia-15][0])
    #print(dia,test_X[dia-14][14][0],yhat[dia-15][0])

grp = plt.figure(figsize=(20,5))
plt.plot(a,"o-",color="blue",label="real")
plt.plot(b,"+-",color="red",label="estimado")
plt.ylabel("cantidad")
plt.title("NO2 en Plaza España (normalizado)")
plt.legend()
plt.show()

In [ ]:
#calcular la desvición típica
print('Desviación típica: ',numpy.std(desvTest))